# Онтологический анализатор нормативных документов
## Генерация и оценка онтологий с использованием LLM

In [ ]:
# Установка зависимостей
!pip install owlready2 openai scikit-learn matplotlib

In [ ]:
import json
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import owlready2
from owlready2 import *
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

## 1. Подготовка тестовых данных

In [ ]:
law_text = """
Закон о персональных данных

Статья 1. Основные понятия
1. Персональные данные - любая информация, относящаяся к прямо или косвенно определенному физическому лицу.
2. Оператор - государственный орган, юридическое лицо, обрабатывающее персональные данные.

Статья 2. Принципы обработки
1. Обработка должна осуществляться на законной основе.
2. Объем данных должен соответствовать заявленным целям.

Статья 3. Права субъекта
1. Субъект имеет право на доступ к своим данным.
2. Субъект может требовать удаления данных.
"""

with open("test_law.txt", "w", encoding="utf-8") as f:
    f.write(law_text)

## 2. Реализация классов системы

In [ ]:
class OntologyGenerator:
    def __init__(self, llm_model="gpt-3.5-turbo"):
        self.llm_model = llm_model
        
    def generate_from_text(self, text, domain):
        # Упрощенная реализация для демонстрации
        onto = get_ontology(f"http://example.com/{domain}.owl")
        
        with onto:
            class ПерсональныеДанные(Thing): pass
            class Оператор(Thing): pass
            class СубъектДанных(Thing): pass
            
            class обрабатывает(ObjectProperty):
                domain = [Оператор]
                range = [ПерсональныеДанные]
                
            class имеет(ObjectProperty):
                domain = [СубъектДанных]
                range = [ПерсональныеДанные]
        
        return onto

In [ ]:
class OntologyEvaluator:
    def __init__(self, ontology_path):
        self.ontology = get_ontology(ontology_path).load()
        
    def calculate_cq_coverage(self, cq_list, answered_cq):
        return len(answered_cq) / len(cq_list)
    
    def visualize_hierarchy(self):
        plt.figure(figsize=(8, 4))
        
        classes = list(self.ontology.classes())
        hierarchy = {}
        
        for cls in classes:
            parents = [p.name for p in cls.is_a if isinstance(p, ThingClass)]
            hierarchy[cls.name] = parents
        
        # Простая визуализация
        for i, (cls, parents) in enumerate(hierarchy.items()):
            plt.text(i, 0, cls, ha='center', va='center', 
                    bbox=dict(facecolor='lightblue', alpha=0.5))
            for parent in parents:
                if parent in hierarchy:
                    j = list(hierarchy.keys()).index(parent)
                    plt.plot([i, j], [0, 1], 'k-')
        
        plt.axis('off')
        plt.title("Иерархия классов онтологии")
        plt.show()

## 3. Генерация онтологии

In [ ]:
generator = OntologyGenerator()
ontology = generator.generate_from_text(law_text, "privacy_law")
ontology.save(file="privacy_law.owl", format="rdfxml")

display(Markdown("### Сгенерированные классы:"))
for cls in ontology.classes():
    display(Markdown(f"- {cls.name}"))

## 4. Визуализация онтологии

In [ ]:
evaluator = OntologyEvaluator("privacy_law.owl")
evaluator.visualize_hierarchy()

## 5. Оценка качества

In [ ]:
cq_list = [
    "Какие сущности обрабатывают персональные данные?",
    "Какие права есть у субъекта данных?"
]

coverage = evaluator.calculate_cq_coverage(cq_list, cq_list[:1])

plt.bar(['Покрытие CQ'], [coverage], color=['lightgreen'])
plt.ylim(0, 1)
plt.title("Оценка покрытия компетентных вопросов")
plt.show()

## 6. Экспорт результатов

In [ ]:
report = {
    "metrics": {
        "cq_coverage": coverage,
        "classes_count": len(list(ontology.classes()))
    },
    "recommendations": [
        "Добавить больше отношений между классами",
        "Увеличить покрытие CQ"
    ]
}

with open("ontology_report.json", "w", encoding="utf-8") as f:
    json.dump(report, f, ensure_ascii=False, indent=2)

display(Markdown("### Отчет сохранен в `ontology_report.json`"))